In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Stock Market Symbol for Cloudflare'}`




/Users/minjaelee/Desktop/3_personal_projects/Gpt/fullstack-gpt/env/lib/python3.11/site-packages/langchain/utilities/duckduckgo_search.py:43: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


Stock Exchange NYSE Ticker Symbol NET Full Company Profile Financial Performance In 2022, Cloudflare's revenue was $975.24 million, an increase of 48.57% compared to the previous year's $656.43 million. Losses were -$193.38 million, -25.71% less than in 2021. Financial Statements Analyst Forecast Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. ... Market . closed. 82.65 +3.06 +3.84 % ... Cloudflare, Inc. - Stock Quote & Chart Stock Quote & Chart Stock Quote NYSE: NET 80.18 Jan 26, 2024 4:00 PM ET Today's Change -1.68 (-2.05%) Today's Volume 2,286,155 Today's Open 81.24 Previous Close 81.86 Intraday High 82.50 Intraday Low 79.80 52 Week High 87.17 52 Week Low 39.90 Pricing delayed by 20 minutes. Last Updated Jan 26, 2024 4:00 PM ET Cloudflare, Inc. is a global cloud services provider that delivers a range of services to businesses of all sizes and in all geographies. The Company's networ

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "The stock market symbol for Cloudflare is NET, and it is listed on the New York Stock Exchange (NYSE).\n\nIn terms of financial performance, in 2022, Cloudflare's revenue was $975.24 million, which is a 48.57% increase compared to the previous year. However, the company had losses of -$193.38 million, which is 25.71% less than in 2021.\n\nAnalysts have provided price targets for Cloudflare's stock. The range of price targets is from $43.00 to $95.00, with an average target of $69.09 for the next year. This suggests that there is a possible downside of 18.1% for the stock.\n\nTo determine if Cloudflare is a potential good investment, it is important to consider various factors such as the company's financial health, growth prospects, competitive landscape, and industry trends. It is recommended to conduct a thorough ana